In [ ]:
class ExtraccionDatos:
    def __init__(self):
        self.info_por_peli = []
        self.lista_tuplas = []
        self.busqueda = busqueda
        self.nombre_archivo = nombre_archivo
    

    def recogida_datos(self, busqueda):
        """ la variable busqueda es la lista de tuplas/listas resultante de la Fase 1"""

        # llegar a web:
        driver = webdriver.Chrome()
        driver.get('https://www.imdb.com/')
        driver.maximize_window()
        sleep(random.randint(1,3))


        # rechazar cookies
        driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
        sleep(random.randint(1,3))

        contador = 0

        # entrando en cada título
        for peli in self.busqueda:
            lista_info = [f'{peli[0]}',]
            contador +=1
            print(f'Película: {contador}')

            try:
                
                # metemos el ID en el buscador
                try:
                    driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
                    sleep(random.randint(1,3))

                    # extraemos info
                    info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
                    lista_info.append(info.split('\n'))

                    self.info_por_peli.append(lista_info)
                except:
                    fallos.append(peli)
                    driver.find_element('css selector', '#error > div.error_attrib > a').click()
                
                            
            except:
                continue    
        
        return self.info_por_peli
    


    def guardar_json(self, nombre_archivo):
        with open(f'{nombre_archivo}.json', 'w') as fichero:
            json.dump(self.info_por_peli, fichero)
        


    def limpiar_datos(self):
        for peli in self.info_por_peli:
    
            patron = re.compile(r'([a-z])([A-Z])')
            
            tupla_datos = [peli[0],] 

            # listas intermedias para ir checkeando
            direccion = []
            guion = []
            nota = []
            duracion = []
            argumento = []
                
            for indice, linea in enumerate(peli[1]):
                    
                
                # directores    
                if 'Dirección' in linea:
                                        
                    directores = patron.sub(r'\1, \2', peli[1][indice+1])
                    direccion.append(directores)            


                # guionistas
                elif 'Guión' in linea:
                    
                    guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
                    guion.append(guionistas)
                        
                
                # nota y duracion
                elif 'PUNTUACIÓN EN IMDb' in linea:
                    nota.append(float(peli[1][indice+1].replace(',', '.')))
                    
                    if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                        duracion.append(peli[1][indice-1])
                    
                            
                # argumento
                elif 'Añade un argumento en tu idioma' in linea:
                    argumento.append(peli[1][indice+1])

            # añadimos a la tupla los datos de la lista intermedia
            for elemento in [direccion, guion, nota, duracion, argumento]:
                if elemento != []:
                    tupla_datos.append(elemento[0])
                else:
                    tupla_datos.append(np.nan)
            
            self.lista_tuplas.append(tupla_datos)

        return self.lista_tuplas
    
    
    def extract_imdb(self):

        self.recogida_datos()

        self.guardar_json()

        self.limpiar_datos()